Taken from https://github.com/serp-ai/bark-with-voice-clone/blob/main/clone_voice.ipynb

In [1]:
#Mount drive to access folder with voice samples to clone from
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
audio_filepath = '/content/drive/MyDrive/pvo/9_10_test.wav' # the audio you want to clone (under 13 seconds)
voice_name = 'output' # whatever you want the name of the voice to be
output_path = '/content/bark-with-voice-clone/bark/assets/prompts/' + voice_name + '.npz'
filepath = "/content/drive/MyDrive/audio_bark_output.wav" # change this to your desired output path

In [3]:
# Enter your prompt
text_prompt = "Deepfakes are AI-generated synthetic videos of any person or celebrity that impersonates the actual person and makes them act or say anything they originally never \
did. The process of creation of deepfakes is technically complex and generally requires a vast amount of data which is then fed to a neural network to train and generate the \
synthetic video. [laughter]"

In [4]:
import time
start=time.time()

In [5]:
# install bark (make sure you have torch>=2 for much faster flash-attention)
# original repository was https://github.com/suno-ai/bark.git
!pip install git+https://github.com/serp-ai/bark-with-voice-clone.git
!git clone https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
!git clone https://github.com/serp-ai/bark-with-voice-clone

  Cloning https://github.com/serp-ai/bark-with-voice-clone.git to /tmp/pip-req-build-fw00a3uv
  Running command git clone --filter=blob:none --quiet https://github.com/serp-ai/bark-with-voice-clone.git /tmp/pip-req-build-fw00a3uv
  Resolved https://github.com/serp-ai/bark-with-voice-clone.git to commit 3159b9a86effb50f8182045f23055551dda8674f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch
import numpy as np

device = 'cuda' # or 'cpu'
model = load_codec_model(use_gpu=True if device == 'cuda' else False)

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:01<00:00, 60.2MB/s]


In [7]:
cd bark-voice-cloning-HuBERT-quantizer/

/content/bark-voice-cloning-HuBERT-quantizer


In [8]:
!pip install -r requirements.txt

Ignoring soundfile: markers 'platform_system == "Windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 31.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.1

In [9]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
from bark_hubert_quantizer.hubert_manager import HuBERTManager
hubert_manager = HuBERTManager()
hubert_manager.make_sure_hubert_installed()
hubert_manager.make_sure_tokenizer_installed()

Downloaded HuBERT


Downloaded tokenizer


'data/models/hubert/tokenizer.pth'

In [10]:
# Load HuBERT for semantic tokens
from bark_hubert_quantizer.pre_kmeans_hubert import CustomHubert
from bark_hubert_quantizer.customtokenizer import CustomTokenizer

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth').to(device)  # Automatically uses the right layers

In [11]:
# Load and pre-process the audio waveform
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

In [12]:
semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

In [13]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [14]:
# move codes to cpu
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

In [15]:
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)

Now you can head over to the generate.ipynb and use your voice_name for the 'history_prompt'. Code from that notebook is pasted below

In [16]:
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

In [17]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True,
    coarse_use_small=False,
    fine_use_gpu=True,
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False,
    path="models"
)

In [18]:
# simple generation
audio_array = generate_audio(text_prompt, history_prompt=output_path, text_temp=0.7, waveform_temp=0.7) #output_path was voice_name

100%|██████████| 35/35 [00:35<00:00,  1.02s/it]


In [19]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=output_path,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=output_path,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=output_path,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

100%|██████████| 34/34 [07:12<00:00, 12.71s/it]


In [20]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [21]:
from scipy.io.wavfile import write as write_wav
# save audio
write_wav(filepath, SAMPLE_RATE, audio_array)

In [22]:
end=time.time()
print("Total time to run:",end-start,'secs')

Total time to run: 969.3935446739197 secs
